In [ ]:
from pace.missingness import *
from pace.history import *
from pace.plots import *

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
# Load helper functions to caluclate IGR
%run 'Information Gain Ratio.ipynb'

In [ ]:
df = pd.read_csv("../../data/Synthetic_APC_DIAG_Fields.csv")

In [ ]:
session = PlotSession(df)

In [ ]:
c = session.missingness().combinations()

c_subset = c[['DIAG_01','DIAG_02','DIAG_03','DIAG_04','DIAG_05','DIAG_06','DIAG_07','DIAG_08','DIAG_09','DIAG_10']]
# plt.matshow(c_subset)

In [ ]:
def has_gap(row: pd.Series) -> bool:
    """Does 'row' have a 'gap' (is it non-monotonic)?"""

    return not row.is_monotonic_increasing

## equivalent to

# def has_gap(row: pd.Series) -> bool:
#    found_missing = False
#    for m in row:
#        found_missing |= m
#        if found_missing and not m:
#            return True
#    return False

In [ ]:
combinations_with_gaps = np.where(c_subset.apply(has_gap, axis=1))[0]
combinations_with_gaps

TODO Explain what's going on here...

In [ ]:
session.add_selection("gaps", combinations=combinations_with_gaps)
gaps = session.selected_records("gaps")
sum(gaps)

In [ ]:
session.add_plot("gaps_plot", "gaps")

In [ ]:
igr_columns = ['ADMIAGE','ADMIMETH','Mortality','PROCODE3','SEX']
igr_result = igr(df[igr_columns], gaps)
igr_result

In [ ]:
plt.bar(igr_result.keys(), igr_result.values())